In [9]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report

### Loading Data

In [10]:
data = np.load('processed_eeg_data.npz')
X_train = data['X_train']
X_val = data['X_val']
y_train = data['y_train']
y_val = data['y_val']

print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(8282, 23, 256) (656, 23, 256) (8282,) (656,)


### Calculating Hurst Scores

These statistics were used by the original baseline model, implemented below

In [4]:
import hurst
from hurst import compute_Hc

In [35]:
# New array to store Hurst exponent and constant for each sample
hurst_train_exp = np.empty((X_train.shape[0], X_train.shape[1]))
hurst_train_const = np.empty((X_train.shape[0], X_train.shape[1]))


# Takes about 5 minutes to run
for i in range(X_train.shape[0]):
    for j in range(X_train.shape[1]):
        hurst_train_exp[i, j], hurst_train_const[i,j], _ = compute_Hc(X_train[i, j], kind='change', simplified=True)
    # Status messages
    if i % 100 == 0:
        print(f"Processed {i} samples")

print(hurst_train_exp.shape, hurst_train_const.shape) # >>> 8282 by 23

Processed 0 samples
Processed 100 samples
Processed 200 samples
Processed 300 samples
Processed 400 samples
Processed 500 samples
Processed 600 samples
Processed 700 samples
Processed 800 samples
Processed 900 samples
Processed 1000 samples
Processed 1100 samples
Processed 1200 samples
Processed 1300 samples
Processed 1400 samples
Processed 1500 samples
Processed 1600 samples
Processed 1700 samples
Processed 1800 samples
Processed 1900 samples
Processed 2000 samples
Processed 2100 samples
Processed 2200 samples
Processed 2300 samples
Processed 2400 samples
Processed 2500 samples
Processed 2600 samples
Processed 2700 samples
Processed 2800 samples
Processed 2900 samples
Processed 3000 samples
Processed 3100 samples
Processed 3200 samples
Processed 3300 samples
Processed 3400 samples
Processed 3500 samples
Processed 3600 samples
Processed 3700 samples
Processed 3800 samples
Processed 3900 samples
Processed 4000 samples
Processed 4100 samples
Processed 4200 samples
Processed 4300 samples


In [36]:
# Same for the validation set
hurst_val_exp = np.empty((X_val.shape[0], X_val.shape[1]))
hurst_val_const = np.empty((X_val.shape[0], X_val.shape[1]))

# 21 seconds to run
for i in range(X_val.shape[0]):
    for j in range(X_val.shape[1]):
        hurst_val_exp[i, j], hurst_val_const[i,j], _ = compute_Hc(X_val[i, j], kind='change', simplified=True)
    # Status messages
    if i % 100 == 0:
        print(f"Processed {i} samples")

print(hurst_val_exp.shape, hurst_train_const.shape) # >>> 656 by 23

Processed 0 samples
Processed 100 samples
Processed 200 samples
Processed 300 samples
Processed 400 samples
Processed 500 samples
Processed 600 samples
(656, 23) (8282, 23)


### Wavelet analysis
Additional stats used by the original baseline


In [13]:
# Taken directly from the original notebook
def statisticsForWavelet(coefs):
    n5 = np.nanpercentile(coefs, 5)
    n25 = np.nanpercentile(coefs, 25)
    n75 = np.nanpercentile(coefs, 75)
    n95 = np.nanpercentile(coefs, 95)
    median = np.nanpercentile(coefs, 50)
    mean = np.nanmean(coefs)
    std = np.nanstd(coefs)
    var = np.nanvar(coefs)
    rms = np.nanmean(np.sqrt(coefs**2))
    return [n5, n25, n75, n95, median, mean, std, var, rms]

In [ ]:
import pywt

train_wavelets = []

# Takes about 9 minutes to run
# Perform wavelet decomposition for each channel in each sample in the training set
for i in range(X_train.shape[0]):
    group = []
    for j in range(X_train.shape[1]):
        # Function returns a list of wavelet coefficients
        # Using 'db4' as in the original notebook
        coefficents = pywt.wavedec(X_train[i, j], 'db4')
        
        subgroup = []
        for k in coefficents:
            # Get statistics for each wavelet decomposition level
            statistics = statisticsForWavelet(k)
            subgroup.append(statistics)
        group.append(subgroup)
    train_wavelets.append(group)
    
    # Status messages
    if i % 100 == 0:
        print(f"Processed {i} samples")


Processed 0 samples
Processed 100 samples
Processed 200 samples
Processed 300 samples
Processed 400 samples
Processed 500 samples
Processed 600 samples
Processed 700 samples
Processed 800 samples
Processed 900 samples
Processed 1000 samples
Processed 1100 samples
Processed 1200 samples
Processed 1300 samples
Processed 1400 samples
Processed 1500 samples
Processed 1600 samples
Processed 1700 samples
Processed 1800 samples
Processed 1900 samples
Processed 2000 samples
Processed 2100 samples
Processed 2200 samples
Processed 2300 samples
Processed 2400 samples
Processed 2500 samples
Processed 2600 samples
Processed 2700 samples
Processed 2800 samples
Processed 2900 samples
Processed 3000 samples
Processed 3100 samples
Processed 3200 samples
Processed 3300 samples
Processed 3400 samples
Processed 3500 samples
Processed 3600 samples
Processed 3700 samples
Processed 3800 samples
Processed 3900 samples
Processed 4000 samples
Processed 4100 samples
Processed 4200 samples
Processed 4300 samples


In [ ]:
train_wavelets = np.array(train_wavelets)
print(train_wavelets.shape)

(8282, 23, 6, 9)


In [ ]:
# Same as above for the validation set
val_wavelets = []


# Will take about 40s to run
# Perform wavelet decomposition for each channel in each sample in the training set
for i in range(X_val.shape[0]):
    group = []
    for j in range(X_val.shape[1]):
        # Function returns a list of wavelet coefficients
        # Using 'db4' as in the original notebook
        coefficents = pywt.wavedec(X_val[i, j], 'db4')
        
        subgroup = []
        for k in coefficents:
            # Get statistics for each wavelet decomposition level
            statistics = statisticsForWavelet(k)
            subgroup.append(statistics)
        group.append(subgroup)
    val_wavelets.append(group)
    
    # Status messages
    if i % 100 == 0:
        print(f"Processed {i} samples")

Processed 0 samples
Processed 100 samples
Processed 200 samples
Processed 300 samples
Processed 400 samples
Processed 500 samples
Processed 600 samples


In [34]:
# Convert lists to numpy arrays
train_wavelets = np.array(train_wavelets)
val_wavelets = np.array(val_wavelets)

print(train_wavelets.shape) # >>> 8282 by 23 by 6 by 9
print(val_wavelets.shape) # >>> 656 by 23 by 6 by 9


(8282, 23, 6, 9)
(656, 23, 6, 9)


### SVM Modeling

In [40]:
# Flatten the wavelet features
train_wavelets = train_wavelets.reshape(train_wavelets.shape[0], train_wavelets.shape[1], -1)
val_wavelets = val_wavelets.reshape(val_wavelets.shape[0], train_wavelets.shape[1], -1)

# Unflatten the Hurst exponent and constant arrays to match the shape of the other features
hurst_train_exp = np.reshape(hurst_train_exp, (-1, 23, 1))
hurst_train_const = np.reshape(hurst_train_const, (-1, 23, 1))

hurst_val_exp = np.reshape(hurst_val_exp, (-1, 23, 1))
hurst_val_const = np.reshape(hurst_val_const, (-1, 23, 1))

print(train_wavelets.shape, hurst_train_exp.shape, hurst_train_const.shape, X_train.shape)
print(val_wavelets.shape, hurst_val_exp.shape, hurst_val_const.shape, X_val.shape)

(8282, 23, 54) (8282, 23, 1) (8282, 23, 1) (8282, 23, 256)
(656, 23, 54) (656, 23, 1) (656, 23, 1) (656, 23, 256)


In [ ]:
# Concatenating the hurst, wavelet, and original features
X_train = np.concatenate(X_train, hurst_train_exp, hurst_train_const, train_wavelets, axis=-1)
X_val = np.concatenate(X_val, hurst_val_exp, hurst_val_const, val_wavelets, axis=-1)


TypeError: stack() got multiple values for argument 'axis'

In [ ]:
print(X_train.shape, X_val.shape)

In [ ]:
# Support Vector Classfier (a type of SVM), uses linear decision boundary
clf = svm.SVC(kernel="linear",probability=True)

# Flatten the datasets to 2D as SVC expects 2D array input
X_train = X_train.reshape(X_train.shape[0], -1)
y_train = y_train.reshape(y_train.shape[0], -1)

# Fits SVM on the training data, 
# I don't know why they are doing this as the next line re-fits the model.
clf.fit(X_train, y_train)

# k-fold cross validation with 10 folds
y_pred = cross_val_predict(clf, X_val, y_val, cv=10)
print("All features are included\n", classification_report(y_val, y_pred))

C:\Users\andre\AppData\Roaming\Python\Python313\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
